In [2]:
!pip install natsort


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for natsort from https://files.pythonhosted.org/packages/ef/82/7a9d0550484a62c6da82858ee9419f3dd1ccc9aa1c26a1e43da3ecd20b0d/natsort-8.4.0-py3-none-any.whl.metadata


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import os
from natsort import natsorted
import joblib
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
# Defining file paths
normal = 'C:\\Users\\pawan\\Downloads\\ultron test\\NORMAL'
abnormal = 'C:\\Users\\pawan\\Downloads\\ultron test\\AHB'
MI = 'C:\\Users\\pawan\\Downloads\\ultron test\\MI'
MI_history = 'C:\\Users\\pawan\\Downloads\\ultron test\\PM'

In [5]:
# Mapping types to file paths
Types_ECG = {'normal': normal, 'Abnormal_hear_beat': abnormal, 'MI': MI, 'History_MI': MI_history}


In [6]:
# Creating lists to store file names
NORMAL_ = []
MI_ = []
PMI_ = []
HB_ = []


In [7]:
# Extracting file names
for types, folder in Types_ECG.items():
    for files in os.listdir(folder):
        if types == 'normal':
            NORMAL_.append(files)
        elif types == 'Abnormal_hear_beat':
            HB_.append(files)
        elif types == 'MI':
            MI_.append(files)
        elif types == 'History_MI':
            PMI_.append(files)

In [8]:
# Sorting file names
NORMAL_ = natsorted(NORMAL_)
MI_ = natsorted(MI_)
PMI_ = natsorted(PMI_)
HB_ = natsorted(HB_)

In [9]:
# Preprocessing and modeling steps
def preprocess_and_model():
    # Looping over and creating combined CSV files for each lead
    for x in range(len(MI_)):
        df1 = pd.read_csv('C:\\Users\\pawan\\Downloads\\ultron test\\NORMAL\\{}'.format(NORMAL_[x]))
        df2 = pd.read_csv('C:\\Users\\pawan\\Downloads\\ultron test\\AHB\\{}'.format(HB_[x]))
        df3 = pd.read_csv('C:\\Users\\pawan\\Downloads\\ultron test\\MI\\{}'.format(MI_[x]))
        df4 = pd.read_csv('C:\\Users\\pawan\\Downloads\\ultron test\\PM\\{}'.format(PMI_[x]))
        final_df = pd.concat([df1, df2, df3, df4], ignore_index=True)
        final_df.to_csv('C:\\Users\\pawan\\Downloads\\ultron test\\Combined_IDLead_{}.csv'.format(x + 1))


In [14]:
    # Reading just lead 1
    df = pd.read_csv(r'C:\Users\pawan\Downloads\ultron test\Combined1d_csv\Combined_IDLead_1.csv')
    df.drop(columns=['Unnamed: 0'], inplace=True)

In [15]:
    # Converting target column values as Numeric using ngroups
    encode_target_label = df.groupby('Target').ngroup().rename("target").to_frame()
    test_final = df.merge(encode_target_label, left_index=True, right_index=True)
    test_final.drop(columns=['Target'], inplace=True)


In [16]:
    # Performing Dimensionality reduction (PCA) on the Dataframe
    pca = PCA(n_components=100)
    x_pca = pca.fit_transform(test_final.iloc[:, 0:-1])
    x_pca = pd.DataFrame(x_pca)


In [17]:
    # Storing the new PCA generated dimensions in a dataframe
    pca_df = pd.DataFrame(data=x_pca)
    target = pd.Series(test_final['target'], name='target')
    result_df = pd.concat([pca_df, target], axis=1)

In [18]:
    # Splitting data into train and test sets
    X = result_df.iloc[:, 0:-1]
    y = result_df.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


In [19]:
    # K-Nearest Neighbors (KNN) Classifier
    knn_pipeline = Pipeline([('knn', KNeighborsClassifier())])
    knn_params = {'knn__n_neighbors': list(range(1, 9))}
    knn_cv = GridSearchCV(knn_pipeline, knn_params, cv=2)
    knn_cv.fit(X_train, y_train)
    knn_accuracy = knn_cv.score(X_test, y_test)


In [22]:
# Logistic Regression Classifier
lr_pipeline = Pipeline([('lr', LogisticRegression())])
lr_params = {'lr__C': np.logspace(-4, 4, 3), 'lr__penalty': ['l2']}
lr_cv = GridSearchCV(lr_pipeline, lr_params, cv=2)
lr_cv.fit(X_train, y_train)
lr_accuracy = lr_cv.score(X_test, y_test)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_